# CDC testing data

* [CDC testing time series](https://healthdata.gov/dataset/covid-19-diagnostic-laboratory-testing-pcr-testing-time-series)

In [ ]:
import pandas as pd
import numpy as np
import pycountry
import json
from datetime import datetime
from functools import reduce
import requests
from io import StringIO
import re
from requests import HTTPError

In [ ]:
LOOKBACK: int = 4

In [ ]:
# papermill parameters

output_folder = "../output/"

In [ ]:
# Get today's path

def get_url(day_offset:int=1) -> str:
    y, m, d = datetime.now().year, datetime.now().month, datetime.now().day
    url = f"https://healthdata.gov/sites/default/files/covid-19_diagnostic_lab_testing_{y}{m:02d}{d-day_offset:02d}_2203.csv"
    return url

In [ ]:
states = [subdivision.code.replace("US-", "") for subdivision in pycountry.subdivisions if subdivision.code.startswith("US-")]

In [ ]:
for i in range(LOOKBACK):
    try:
        df = pd.read_csv(get_url(i))
        break
    except Exception:
        continue

In [ ]:
df = df[["state", "overall_outcome", "date", "new_results_reported"]].pivot(index=["state", "date"], values=["new_results_reported"], columns=["overall_outcome"])
df = pd.DataFrame(df.to_records())
df.columns = ["ISO3166_2", "DATE", "INCONCLUSIVE", "POSITIVE", "NEGATIVE"]
df["ISO3166_1"] = "US"
df.to_csv(output_folder + "CDC_TESTING.csv", index=False)